### 주가정보 획득
* 마케팅 포인트와 주가와의 상관관계 분석을 위한 주가 획득
* 유통주 위주의 주가분석
* 현대백화점(069960)  
 - https://finance.naver.com/item/sise_day.nhn?code=069960&page=1


In [1]:
import sqlite3
import requests # 내부 source 를 가져오는 라이브러리
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

C:\Users\user\anaconda3\envs\py36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# 함수 선언부 url 을 매개변수화
def stockGet(url=''):    
    # url 가져와서 파싱
    src=requests.get(url)
    result=src.content.strip()
    html=BeautifulSoup(result,'html.parser')
    tgts=html.select('table.type2 tr')
    #len(tgt)
    res=[]
    #tgts 가 16개 씩 나옴 1 page
    for tgt in tgts:
        if(tgt.select('td .p10')):# 날짜 없는 줄은 제거
            sday=tgt.select('td .p10')[0].text.replace(".","")

            endp=tgt.select('td.num')[0].text.replace(",","")
            startp=tgt.select('td.num')[2].text.replace(",","")
            maxp=tgt.select('td.num')[3].text.replace(",","")
            minp=tgt.select('td.num')[4].text.replace(",","")
            vol=tgt.select('td.num')[5].text.replace(",","")
            res.append((sday,startp,maxp,minp,endp,vol))
    # 10개 반환
    return res


In [3]:
def addData(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql = 'insert into StockDay \
        (SIDX,SCODE,SCOMP,SDAY,OPENP,HIGHP,LOWP,CLOSEP,VOL) \
        values (?,?,?,?,?,?,?,?,?)'
        ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        cur.execute(sql,data)
        conn.commit()
        cur.close()
    
    

In [4]:
dbname='NaverStock.db'
def selData(whr=' where SIDX=0'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select SIDX from StockDay '+whr
        cur.execute(sql)
        rows=cur.fetchall()
        cur.close()
    return rows

In [5]:
compList=[
    ("현대백화점","069960"),
    ("영원무역","111770"),
    ("SK네트웍스","001740"),
    ("신세계인터내셔널","031430"),
    ]

In [6]:
def MyCrawl(comp=('현대백화점','069960'),sp=1,ep=2):
    sCode=comp[1]
    sComp=comp[0] 
    urlsrc='https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
    for i in range(1,2):
        print("#"*30)
        url=urlsrc.format(sCode,i)
        print(url)
        rs=stockGet(url)
        #10개 데이터 추출
        for r in rs:
            #데이터별 고유코드만들기
            sidx=r[0]+sCode
            ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
            data=[sidx,sCode,sComp,r[0],r[1],r[2],r[3],r[4],r[5]]
            #데이터 입력하기 중복데이터 확인
            preData=selData(' where SIDX='+sidx)
            if(preData):
                print("중복데이터 입니다.",preData)
            else:
                #print(r)
                addData(data)

In [7]:
MyCrawl(compList[0],1,2)

##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=1


OperationalError: no such table: StockDay

In [8]:
selData()

OperationalError: no such table: StockDay